In [ ]:
import locale

locale.setlocale(locale.LC_ALL, "de_AT.UTF-8")

%matplotlib inline
%precision %.4f
# %load_ext snakeviz
import colorcet
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.dates
import matplotlib.ticker
import matplotlib.patches
import matplotlib.colors
import seaborn as sns
from importlib import reload
from itertools import count
from datetime import date, datetime, timedelta
from IPython.display import display, Markdown, HTML
import textwrap
from functools import partial
from cycler import cycler
import collections

from covidat import cov, util

pd.options.display.precision = 4

cov = reload(cov)

plt.rcParams['figure.figsize'] = (16 * 0.7, 9 * 0.7)
plt.rcParams['figure.dpi'] = 120  # 80
plt.rcParams['figure.facecolor'] = '#fff'
sns.set_theme(style="whitegrid")
sns.set_palette('tab10')

plt.rcParams['image.cmap'] = cov.un_cmap

pd.options.display.max_rows = 120
pd.options.display.min_rows = 40

In [ ]:
def stampit(fig):
    cov.stampit(fig, "Statistik Austria")

In [ ]:
if False:
    st = pd.read_csv("sterbetafel-at.csv", sep=";", encoding="utf-8", header=[0, 1], decimal=",", index_col=0)
    st = st.melt(ignore_index=False, var_name=["sex", "age"], value_name="life_rem")
    st.index = st.index.rename("year")
    st["age"] = st["age"].astype(int)
    st["life_sum"] = st["life_rem"] + st["age"]
    st.set_index(["sex", "age"], append=True, inplace=True)

In [ ]:
def plt_sterbe_simp(st0, title):
    # sns.lineplot()
    rwidth = 1
    change = (
        st0["life_sum"].groupby(level=list(set(st0.index.names) - {"year"}))
        # .transform(lambda s: s.rolling(rwidth).mean() - s.rolling(rwidth).mean().shift(rwidth))
        .transform(lambda s: s - s.shift(rwidth))
    )
    maxage = st0.index.get_level_values("age").max()
    ag60 = st0.xs(maxage, level="age")
    spread = (ag60["life_sum"] / st0.query(f"age != {maxage}")["life_sum"]).rename("exp_spread")
    # stpiv = st0.reset_index().pivot(index="year", columns="age", values="life_sum")
    # spread = (stpiv.loc[:, 60] / stpiv.loc[:, 45]).rename("exp_spread")
    # display(spread)
    # .transform(lambda s: s.rolling(rwidth).mean() - s.rolling(rwidth).mean().shift(rwidth))

    # display(spread)    #change = change[change.index.get_level_values("year") >= 1950]
    # st0 = st0[st0.index.get_level_values("year") >= 1950]
    # display(st0)
    # print(change.first_valid_index())
    # display(change[change.index.get_level_values("age") == 0].tail(12))
    plt.figure()
    ax = sns.lineplot(
        change.reset_index(), x="year", y="life_sum", hue="age", palette="Paired", marker=".", alpha=0.8, mew=0
    )
    fig = ax.figure
    fig.suptitle(
        title
        + (
            # f": Änderung des {rwidth}-Jahre-Schnitts ggü. {rwidth} Jahren zuvor"
            f": Änderung ggü. {rwidth} Jahren zuvor"
            if rwidth != 1
            else ": Änderung ggü. Vorjahr"
        ),
        y=0.93,
    )
    ax.legend(title="Erreichtes Alter", ncol=2)
    ax.set_ylabel("Jährliche Änderung Lebenserwartung")
    ax.set_xlabel("Jahr")
    # print(st0.iloc[0].name)
    ax.set_xlim(left=change.index[0][change.index.names.index("year")])
    # ax.axvspan(2019.5, ax.get_xlim()[1], alpha=0.5, color="yellow")
    ax.axhline(0, color="k")

    plt.figure()
    ax = sns.lineplot(st0.reset_index(), x="year", y="life_sum", hue="age", palette="Paired", marker=".", mew=0)
    ax.figure.suptitle(title, y=0.93)
    cov.labelend2(ax, st0.reset_index(), "life_sum", cats="age", x="year", shorten=lambda n: n)
    ax.set_xlim(left=st0.index.get_level_values("year")[0])
    ax.legend(title="Erreichtes Alter", ncol=2)
    ax.set_ylabel("Gesamte Lebenserwartung")
    ax.set_xlabel("Jahr")

    plt.figure()
    ax = sns.lineplot(spread.reset_index(), x="year", y="exp_spread", hue="age", palette="Paired", marker=".", mew=0)
    ax.figure.suptitle(title + f": Verhältnis zu {maxage}-jährigen", y=0.93)
    ax.set_xlim(left=st0.index.get_level_values("year")[0])
    ax.legend(title="Erreichtes Alter", ncol=2)
    ax.set_ylabel(f"Verhältnis Lebenserwartung zu bereits {maxage}-jährigen")
    ax.set_xlabel("Jahr")
    ax.set_ylim(bottom=1)
    cov.set_percent_opts(ax, decimals=1)


if False:
    plt_sterbe_simp(st.query("sex == 'M'"), "Österreich: Lebenserwartung Männer")
    plt_sterbe_simp(st.query("sex == 'F'"), "Österreich: Lebenserwartung Frauen")
    plt_sterbe_simp(st.groupby(level=["year", "age"]).mean(), "Österreich: Lebenserwartung Schnitt M+W")

In [ ]:
bcols = ["p_die", "n_surv", "n_die", "n_at", "n_from", "life_rem"]


def mkcols(pfx):
    return [pfx + "_" + c for c in bcols]


cols_old = ["age"] + mkcols("m") + mkcols("f")
cols_new = cols_old + mkcols("mf")
wks = pd.read_excel(
    util.COLLECTROOT / "Jaehrliche_Sterbetafeln_1947_bis_2022_fuer_Oesterreich.ods",
    sheet_name=list(map(str, range(1947, 2002))),
    skiprows=12,
    usecols=range(len(cols_old)),
    header=None,
    names=cols_old,
    nrows=96,
    index_col=None,
)
wks.update(
    pd.read_excel(
        util.COLLECTROOT / "Jaehrliche_Sterbetafeln_1947_bis_2022_fuer_Oesterreich.ods",
        sheet_name=list(map(str, range(2002, 2022 + 1))),
        skiprows=7,
        usecols=range(len(cols_new)),
        header=None,
        names=cols_new,
        nrows=101,
        index_col=None,
    )
)

In [ ]:
for year, sheet in wks.items():
    sheet.dropna(inplace=True, how="all")
    sheet["year"] = int(year)
ststat = pd.concat(wks.values())
ststat["age"] = ststat["age"].astype(int)
ststat.set_index(["year", "age"], inplace=True)
ststat.columns = ststat.columns.str.split('_', n=1, expand=True)
ststat.columns.set_names(["sex", None], inplace=True)
ststat = ststat.stack(level=0)
ststat = ststat.reorder_levels(["year", "sex", "age"]).sort_index()
# lastyear = ststat.reset_index("age")["age"].groupby(["year", "sex"]).max()
ststat.loc[ststat["n_at"] == ststat["n_from"], "n_at"] = np.nan
ststat_amf = ststat.groupby(["year", "age"]).mean()
ststat_amf["sex"] = "amf"
ststat = pd.concat(
    [ststat, ststat_amf.set_index("sex", append=True).reorder_levels(["year", "sex", "age"])]
).sort_index()
# display(ststat.index.dtypes)
# display(ststat_amf)
ststat["life_sum"] = ststat.index.get_level_values("age") + ststat["life_rem"]

In [ ]:
def _try_cross():
    year = ststat.index.get_level_values("year").max()

    # p_surv = (1 - ststat.query("age <= 40")["p_die"]).xs((year, "m")).rename("p_surv")
    # display(n_surv)
    # p_survx = p_surv * p_surv.transpose()
    def calc_survmat(year):
        n_surv = ststat.query("age <= 99").xs((year, "m"))["n_surv"]
        survx = n_surv.to_frame().dot((1 / n_surv).to_frame().transpose())
        survx.columns = survx.columns.copy()
        survx.columns.name = "age_from"
        survx.index.name = "age_to"
        trimask = np.ones(survx.shape, dtype='bool')
        trimask[np.tril_indices(len(survx))] = False
        survx.mask(trimask, inplace=True)
        survx = survx.clip(upper=1)
        survx = 1 - survx
        mask = np.zeros(survx.shape, dtype="bool")
        mask[np.diag_indices(len(survx))] = True
        survx[mask] = 0
        return survx

    survx = calc_survmat(year) - calc_survmat(2019)
    # display(survx)
    # survx = survx.where(lambda x: x <= 1)

    # survx = survx.sort_index(ascending=False)
    # display(survx)
    fig, ax = plt.subplots()
    vmin = 0  # 0.0025
    # .where(lambda x: x >= vmin, vmin).mask(trimask)
    im = ax.imshow(
        survx.T,
        aspect=1,
        cmap=cov.div_l_cmap,
        interpolation='none',
        # norm=matplotlib.colors.LogNorm(vmin=0.0025, vmax=0.05),
        norm=matplotlib.colors.TwoSlopeNorm(vmin=-0.001, vcenter=0, vmax=0.05),
        origin="lower",
    )
    cax = fig.colorbar(im)
    # cov.set_logscale(cax.ax)
    cov.set_percent_opts(cax.ax, decimals=1)
    fig.suptitle(f"{year}: Sterbewahrscheinlichkeit zwischen Altern")
    ax.set_xlabel("Zu erreichendes Alter")
    ax.set_ylabel("Bereits erreichtes Alter")


_try_cross()

In [ ]:
survfrom1 = (
    (1 - ststat.query("age >= 5")["p_die"])
    .groupby(level=["year", "sex"])
    .transform(lambda s: s.cumprod())
    .rename("p_reachend")
)
survfrom1.xs((2019, 'f'))


def plt_surv(survdata):
    ax = sns.lineplot(survdata.reset_index(), x="year", y="p_reachend", hue="age", palette="tab10", marker=".", mew=0)
    cov.set_logscale(ax)
    ax.set_ylim(bottom=0.04, top=0.55)
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.05))


plt_surv((1 - survfrom1).reset_index().query("sex == 'amf' and age in (60, 70, 80, 85)"))

In [ ]:
ax = sns.lineplot(
    ststat.reset_index().query("year >= 2019 and sex == 'mf' and 0 <= age <= 85"),
    x="age",
    y="p_die",
    hue="year",
    palette="tab10",
    # markers="year",
    marker=".",
    mew=0,
)
cov.set_logscale(ax)

In [ ]:
if False:
    n_die5 = (1 - p_surv5) * ststat["n_surv"]
    survchange = n_die5.groupby(level=["sex", "age"]).transform(lambda s: s - s.shift().rolling(5).min())
elif False:
    n_surv5 = ststat["n_surv"]

    def lookup_maxsurv(s):
        try:
            return n_surv5.loc[(maxrem.loc[s[: maxrem.index.nlevels]], *s[maxrem.index.nlevels :])]
        except KeyError:
            return np.nan

    def calc_survchange(s: pd.Series):
        n_surv5.loc[s.name]
        best5 = s.index.map(lookup_maxsurv)
        return s - best5

    # display(n_surv5best)
    survchange = (
        n_surv5.groupby(level=["year", "sex"]).join(maxsurv)
        # .transform(lambda s: s - s.shift().rolling(5).max())
    )
# survchange *= ststat["n_surv"]
# survchange.rename("p_surv5", inplace=True);

In [ ]:
def sexlabel(sex: str) -> str:
    return 'Männer' if sex == 'm' else 'Frauen' if sex == 'f' else 'Schnitt M/W' if sex == 'amf' else '?!'


def plt_year_comp(pltdata, sex, mainlabel):
    # surv5.xs((2019, 'm')).plot()
    _sex = sex
    data = pltdata.xs(_sex, level="sex").rename("y").reset_index().query("0 <= age")
    nyear = data["year"].nunique()
    ax = plt.subplot()
    fig = ax.figure
    ndetail = 6
    detailyear = data["year"].max() - (ndetail - 1)
    palette = sns.color_palette("tab10", n_colors=ndetail)[-nyear:]
    closeyear = 1975
    predetail = data.query(f"year < {detailyear}")
    predetail_close = predetail.query(f"year >= {closeyear}")

    def _nicelabels(data):
        data = data.copy()
        data["year"] = data["year"].map(lambda yr: f"{yr} vs. {maxrem.loc[(yr, _sex)]}")
        return data

    detail = data.query(f"year >= {detailyear}")
    sns.lineplot(
        _nicelabels(detail),
        ax=ax,
        x="age",
        hue="year",
        y="y",
        # palette=palette,
        palette="tab10",
        # palette="cet_glasbey_bw",
        marker=".",
        mew=0,
    )
    sns.lineplot(
        predetail_close,
        ax=ax,
        x="age",
        y="y",
        # palette="cet_glasbey_bw",
        errorbar=("pi", 95),
        lw=0,
        mew=0,
        color="C0",
        err_kws=dict(label=f"95. Perzentil {closeyear}‒{detailyear - 1}"),
    )
    ax.set_ylim(*ax.get_ylim())
    sns.lineplot(
        predetail.loc[~predetail["year"].isin(predetail_close["year"])],
        ax=ax,
        x="age",
        y="y",
        # palette="cet_glasbey_bw",
        # errorbar=("pi", 95),
        units="year",
        estimator=None,
        mew=0,
        color="C1",
        alpha=0.1,
        zorder=0,
    )

    sns.lineplot(
        predetail,
        ax=ax,
        x="age",
        y="y",
        # palette="cet_glasbey_bw",
        errorbar=("pi", 95),
        lw=0,
        mew=0,
        color="C1",
        err_kws=dict(
            label=f"95. Perzentil {data.loc[data['y'].first_valid_index(), 'year']}‒{detailyear - 1}",
            alpha=0.07,
        ),
    )

    sns.lineplot(
        predetail_close,
        ax=ax,
        x="age",
        y="y",
        # palette="cet_glasbey_bw",
        # errorbar=("pi", 95),
        units="year",
        estimator=None,
        mew=0,
        color="C0",
        alpha=0.2,
        zorder=0,
    )
    # cov.set_logscale(ax)
    # ax.yaxis.set_major_locator(matplotlib.ticker.AutoLocator())
    ax.legend(ncol=5, fontsize="x-small")
    # ax.set_ylim(bottom=0.95)
    ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(5))
    ax.axhline(1, color="lightgrey", lw=2)
    cov.labelend2(
        ax,
        detail,
        "y",
        cats="year",
        x="age",
        ends=(True, True),
        shorten=lambda n: n,
        fontsize="small",
        colorize=palette,
    )
    # ax.set_ylim(top=1.002, bottom=0.998)

    # cov.set_percent_opts(ax)

    fig.suptitle(f"{sexlabel(sex)}:" f" {mainlabel} vs. Vorjahr mit max. Lebenserwartung")
    ax.set_title(
        "Bezogen auf eine fiktive \"Sterbetafelbevölkerung\" in"
        " der konstant jährlich gleich viele Lebendgeborene hinzukommen"
        "\nDatenquelle: Sterbetafeln der Statistik Austria | Darstellung & weitere Berechnung: @zeitferne"
    )
    return fig, ax


# ax.set_xlim(left=-1, right=50)
# ax.set_ylim(top=1.5)
# display(ststat.xs("m", level="sex").xs(38, level="age"))
# display(data.query("age == 63"))

In [ ]:
maxrem = (
    ststat.xs(1, level="age")["life_rem"]
    .groupby(level="sex")
    .transform(lambda g: g.rolling(5).agg(lambda s: s.idxmax()[0]).shift())
    .dropna()
    .astype(int)
).rename("maxrem_idx")


def join_maxrem(df):
    return df.join(maxrem).merge(df, left_on=["maxrem_idx", "sex", "age"], right_index=True, suffixes=(None, '_max'))

In [ ]:
n_die_s = (
    ststat["n_die"]
    .groupby(["year", "sex"])
    .rolling(pd.api.indexers.FixedForwardWindowIndexer(window_size=15))
    .sum()
    .reset_index([0, 1], drop=True)  # .query("age <= 95")["n_at"].copy()
)
p_die_s = (n_die_s / ststat["n_surv"]).rename("p_die_s")
# n_at.loc[n_at.index.get_level_values("age") == 95] = ststat["n_from"]
p_die_s_or = p_die_s / join_maxrem(p_die_s.to_frame())["p_die_s_max"]


def plt_die_or(sex):
    fig, ax = plt_year_comp(p_die_s_or, sex, "Relative Sterbewahrscheinlichkeit/15 Jahre")
    ax.set_ylabel("Relative Änderung Sterbewahrscheinlichkeit innert 15 J (OR)")
    ax.set_xlabel("Erreichtes Lebensalter")


for sex in ("m", "f", "amf"):
    plt.figure()
    plt_die_or(sex)

In [ ]:
ststat.reset_index()["year"].min()

In [ ]:
ststat.query("year == 2021")

In [ ]:
pdata = ststat.query("year in (1947, 1949, 1980, 2021, 2019) and sex == 'm'").copy()
# display(pdata)
# pal = sns.color_palette("plasma_r", n_colors=pdata["year"].nunique())
pal = sns.color_palette(n_colors=5)  # ["violet", "pink"] + sns.color_palette("Blues", 2) + sns.color_palette("Reds", 2)
pdata["pdie"] = 1 - pdata["n_surv"] / pdata.xs(0, level="age")["n_surv"]
pdata["age_from"] = 0
pdata_y = pdata.copy()
pdata_y["pdie"] = 1 - pdata_y["n_surv"] / pdata_y.xs(20, level="age")["n_surv"]
pdata_y["age_from"] = 20
pdata_y.set_index("age_from", append=True, inplace=True)
pdata_y = pdata_y.query("age > age_from")

pdata_y2 = pdata.copy()
pdata_y2["pdie"] = 1 - pdata_y2["n_surv"] / pdata_y2.xs(45, level="age")["n_surv"]
pdata_y2["age_from"] = 45
pdata_y2.set_index("age_from", append=True, inplace=True)
pdata_y2 = pdata_y2.query("age > age_from")
pdata.set_index("age_from", append=True, inplace=True)
pdata = pdata.query("age > 0")
pdata = pd.concat([pdata, pdata_y, pdata_y2])

pdata.reset_index(inplace=True)
ax = sns.lineplot(pdata, x="age", y="pdie", hue="year", style="age_from", palette=pal)
fig = ax.figure
# cov.labelend2(ax, pdata, "life_sum", cats="year", x="age", shorten=lambda y: y, colorize=pal, ends=(True, True))
# ax.legend(ncol=2)
ax.set_ylabel("Anteil der Lebendgeborenen die vorher versterben")
ax.set_title("(Niedriger ist besser)")
ax.set_xlabel("Zu erreichendes Alter")
# ax.set_ylim(bottom=81)
# ax.set_ylim(bottom=0)
cov.set_logscale(ax)
ax.set_ylim(top=1)
cov.set_percent_opts(ax, decimals=2)
# ax.set_ylim(bottom=0.01)

fig.suptitle("Österreich: Wahrscheinlichkeit vor Erreichen eines Alters zu sterben (Schnitt M/W)", y=0.95)
# ax.set_title("(Y-Achse beginnt nicht bei 0!)")

In [ ]:
ststat.query("sex == 'mf'").reset_index()["year"].unique()

In [ ]:
pdata = ststat.query("year in (2021, 2020, 2019, 2014) and sex == 'mf'").reset_index()
# pal = sns.color_palette("plasma_r", n_colors=pdata["year"].nunique())
pal = sns.color_palette("Blues", 2) + sns.color_palette("Reds", 2)
ax = sns.lineplot(pdata, x="age", y="life_sum", hue="year", palette=pal)
fig = ax.figure
# cov.set_logscale(ax)
cov.labelend2(ax, pdata, "life_sum", cats="year", x="age", shorten=lambda y: y, colorize=pal, ends=(True, True))
ax.legend(ncol=2, title="Kalenderjahr")
ax.set_ylabel("Gesamte Lebenserwartung")
ax.set_xlabel("Bereits erreichtes Alter")
ax.set_ylim(bottom=81)
fig.suptitle("Österreich: Lebenserwartung (M+F) je Kalenderjahr und erreichtem Alter", y=0.95)
ax.set_title("(Y-Achse beginnt nicht bei 0!)")

In [ ]:
def plt_p_die_cmap(sex):
    fig, ax = plt.subplots()
    pdata = (
        ststat.xs(sex, level="sex").query("age <= 85").reset_index().pivot(index="age", columns="year", values="p_die")
    )
    # pdata = pdata / pdata.shift(axis="columns")
    im = ax.imshow(
        pdata,
        cmap=cov.un_cmap,
        interpolation='none',
        origin="lower",
        aspect="auto",
        extent=[
            pdata.columns[0],
            pdata.columns[-1] + 1,
            pdata.index[0],
            pdata.index[-1] + 1,
        ],
        # norm=matplotlib.colors.TwoSlopeNorm(vmin=0.75, vcenter=1, vmax=1.1),
        norm=matplotlib.colors.LogNorm(vmin=0.0003, vmax=0.1),
    )
    ax.grid(False)
    cax = fig.colorbar(im)
    cov.set_logscale(cax.ax, reduced=True)
    cov.set_percent_opts(cax.ax, decimals=2)
    ax.set_ylabel("Lebensalter")
    ax.set_xlabel("Kalenderjahr")
    ax.set_title(
        sexlabel(sex) + ": Sterbewahrscheinlichkeit innerhalb eines Jahres" + f" {pdata.columns[0]}‒{pdata.columns[-1]}"
    )
    ax.tick_params(left=True, bottom=True)


plt_p_die_cmap("m")
plt_p_die_cmap("f")

In [ ]:
def plt_lexp():
    stat0 = ststat.xs(0, level="age")  # .query("year >= 2000")
    fig, ax = plt.subplots()
    cats = [("m", "C0"), ("amf", "darkgrey"), ("f", "C2")]
    cats.reverse()
    for sex, cl in cats:
        stat0x = stat0.xs(sex, level="sex")["life_sum"]
        is_worsened = stat0x < stat0x.shift()
        ax.plot(
            stat0x.where(is_worsened),
            marker="o",
            color="r",
            markersize=8,
            lw=5,  # alpha=0.7,
            label="Verschlechterung" if sex is cats[0][0] else None,
        )
        ax.plot(stat0x, marker=".", label=sexlabel(sex), color=cl)
        ax.annotate(r"Ø" if sex == "amf" else sex, (stat0x.index[-1] + 1, stat0x.iloc[-1]), va="center")
    fig.suptitle(f"Österreich: Lebenserwartung ab Geburt im Verlauf der Jahre je Geschlecht", y=0.93)
    ax.legend()
    # cov.set_percent_opts(ax)
    ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(5))
    ax.set_xlabel("Kalenderjahr")
    ax.set_ylabel("Lebenserwartung ab Geburt (Jahre)")
    stampit(fig)


plt_lexp()

In [ ]:
def plt_die50(relative=False):
    stat0 = ststat  # .xs("f", level="sex")#.query("year >= 2000")
    tgtage = 40
    fromage = 1
    nsurv50 = stat0.xs(tgtage, level='age')["n_surv"]
    psurv50 = nsurv50 / stat0.xs(fromage, level='age')["n_surv"].rename("p_surv_rng")
    if relative:
        psurv50 = 1 - psurv50
    fig, ax = plt.subplots()
    cats = [("m", "C0"), ("amf", "darkgrey"), ("f", "C2")]
    if not relative:
        cats.reverse()
    for sex, cl in cats:
        psurv50x = psurv50.xs(sex, level="sex")
        if relative:
            is_worsened = psurv50x > psurv50x.shift()
        else:
            is_worsened = psurv50x < psurv50x.shift()
        ax.plot(
            psurv50x.where(is_worsened),
            marker="o",
            color="r",
            markersize=8,
            lw=5,  # alpha=0.7,
            label="Verschlechterung" if sex is cats[0][0] else None,
        )
        ax.plot(psurv50x, marker=".", label=sexlabel(sex), color=cl)
        ax.annotate(r"Ø" if sex == "amf" else sex, (psurv50x.index[-1] + 1, psurv50x.iloc[-1]), va="center")
    if relative:
        cov.set_logscale(ax)
        ax.yaxis.set_major_locator(matplotlib.ticker.LogLocator(base=10, subs=[0.15, 0.25, 0.5, 0.7, 1]))
    else:
        ax.set_ylim(top=1)
    fig.suptitle(
        f"Österreich: Wahrscheinlichkeit, ab Lebensjahr {fromage} das {tgtage}. nicht zu erreichen (logarithmische Skala)"
        if relative
        else f"Österreich: Wahrscheinlichkeit dass ein/e {fromage}-jährige/r, {tgtage} Jahre oder älter wird",
        y=0.93,
    )
    ax.legend()
    cov.set_percent_opts(
        ax, decimals=1 if psurv50.min() < 0.01 or psurv50.max() > 0.99 or psurv50.min() < 3 and relative else 0
    )
    ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(5))
    ax.set_xlabel("Kalenderjahr")
    ax.set_ylabel("Sterbewahrscheinlichkeit (log)" if relative else "Überlebenswahrscheinlichkeit")
    stampit(fig)


plt_die50(True)
plt_die50(False)